In [ ]:
#imports
import os
import shutil
import tensorflow as tf
import numpy as np
from PIL import Image
import cv2
import random
import Augmentor
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img


2025-02-12 22:18:47.420949: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-12 22:18:47.430332: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739369927.441072   27540 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739369927.443930   27540 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-12 22:18:47.455315: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
#import directory
directory = open(dataPath + "ascii/words.txt","r").read().splitlines()
len(directory)
directory = directory[18:] #first 18 lines are instructions


In [ ]:
# import images
#--- words.txt ---------------------------------------------------------------#
#
# iam database word information
#
# format: a01-000u-00-00 ok 154 1 408 768 27 51 AT A
#
#     a01-000u-00-00  -> word id for line 00 in form a01-000u
#     ok              -> result of word segmentation
#                            ok: word was correctly
#                            er: segmentation of word can be bad
#
#     154             -> graylevel to binarize the line containing this word
#     1               -> number of components for this word NOTE: dont think this exists idk why
#     408 768 27 51   -> bounding box around this word in x,y,w,h format
#     AT              -> the grammatical tag for this word, see the
#                        file tagset.txt for an explanation
#     A               -> the transcription for this word
#
class img:
    def __init__ (this,data):
        data = data.split(" ")
        this.d = data
        this.id = data[0]
        this.parts = data[0].split("-")
        a = this.parts
        this.loc = a[0]+"/"+a[0]+"-"+a[1]+"/"+data[0]
        this.path = dataPath + "words/" + this.loc
        this.word = data[-1]
    def augPath(this,s):
        return dataPath + s + this.loc

images = []
for i in directory:
    images.append(img(i))
print(len(images))





115320


In [ ]:
#To add a pixel of border around each picture, because zooming out uses the edge pixels
def addBorder (path):
    t = cv2.imread(path+".png")
    pic = np.array(t)
    x,y,z = pic.shape #ALSO change colour here if needed

    column = np.array([[[255,255,255]] for i in range (x)]) # to add 1 pixel of white around the picture, because the picture is cropped to exactly where the side contains black and zoom function does not take that well
    row = np.array([[[255,255,255] for i in range (y+2)]])
    pic = np.concatenate([column,pic,column], axis=1)
    pic = np.concatenate([row,pic,row],axis=0)
    cv2.imwrite(path+".jpg",pic)
    return pic

for img in images:
    try:
        addBorder(img.path)
    except:
        print(img.loc)

[ WARN:0@0.617] global loadsave.cpp:848 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.


a01/a01-117/a01-117-05-02
r06/r06-022/r06-022-03-05


In [ ]:
#create Directory for augmented Images
def createFile (path):
    if not os.path.exists(path):
        os.makedirs(path)
for img in images:
    createFile(img.augPath("augmentedWords/"))

In [ ]:
#define augment function
#https://www.geeksforgeeks.org/python-data-augmentation/
def augment1 (path,savePath):
    # Converting the input sample image to an array
    # Reshaping the input image
    datagen = ImageDataGenerator(zoom_range = [1,2.5],rotation_range = 15, horizontal_flip=True, vertical_flip=True)
   
    # Loading a sample image 
    img = load_img(path) 
    # Converting the input sample image to an array
    x = img_to_array(img)
    # Reshaping the input image
    x = x.reshape((1, ) + x.shape) 
    
    # Generating and saving 5 augmented samples 
    # using the above defined parameters. 
    i = 0
    for batch in datagen.flow(x, batch_size = 1,
                            save_to_dir =savePath, 
                            save_prefix ='image', save_format ='jpeg'):
        i += 1
        if i >= 10:
            break
def augment2 (path,savePath):
    # Converting the input sample image to an array
    # Reshaping the input image
    datagen = ImageDataGenerator(zoom_range = [1,2],rotation_range = 30,width_shift_range=0.1,height_shift_range=0.1)
   
    # Loading a sample image 
    img = load_img(path) 
    # Converting the input sample image to an array
    x = img_to_array(img)
    # Reshaping the input image
    x = x.reshape((1, ) + x.shape) 
    
    # Generating and saving 5 augmented samples 
    # using the above defined parameters. 
    i = 0
    for batch in datagen.flow(x, batch_size = 1,
                            save_to_dir =savePath, 
                            save_prefix ='image', save_format ='jpeg'):
        i += 1
        if i >= 10:
            break

In [ ]:
#execute augment function, will take long time
#change
last = ""
for img in images:
    try:
        if img.parts[0] != last:
            last = img.parts[0]
            print(last)
        if len(img.word) > 1:
            augment1(img.path+".jpg",img.augPath("augmentedWords/"))
        else:
            augment2(img.path+".jpg",img.augPath("augmentedWords/"))
    except:
        print(img.loc)

a01
a01/a01-117/a01-117-05-02
a02
a03
a04
a05
a06
b01
b02
b03
b04
b05
b06
c01
c02
c03
c04
c06
d01
d03
d04
d05
d06
d07
e01
e02
e04
e06
e07
f01
f02
f03
f04
f07
g01
g02
g03
g04
g05
g06
g07
h01
h02
h04
h05
h06
h07
j01
j04
j06
j07
k01
k02
k03
k04
k07
l01
l03
l04
l07
m01
m02
m03
m04
m06
n01
n02
n03
n04
n06
p01
p02
p03
p06
r02
r03
r06
r06/r06-022/r06-022-03-05


In [ ]:
print(len(images))

115320


In [ ]:

for img in images:
    createFile(img.augPath("final_aug_words/"))

NameError: name 'createFile' is not defined

In [ ]:
def resize(image_path,size,out_path): #resize to size and add black padding around unknown
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, 1)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize_with_pad(image, size[0],size[1])
    tf.keras.utils.save_img(out_path, image, file_format="jpeg")
last = ""
for img in images:
    try:
        if img.parts[0] != last:
                last = img.parts[0]
        files = os.listdir(img.augPath("augmentedWords/"))
        endpath = img.augPath("final_aug_words/")
        
        for augImg in files:
            resize (img.augPath("augmentedWords/")+"/"+augImg,(224,224),endpath+"/"+augImg)
    except:
        print (img.loc)
#takes 46 mins

I0000 00:00:1739369999.075536   27540 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9533 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1
